In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [2]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-6,
    'BATCH_SIZE':64,
    'SEED':41
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [4]:
LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'
PATH = os.getcwd() + '/data/'

## About Features
#### New_Train : train_data after feature engineering
O : 학습에 사용될 Features
X : 학습에서 drop할 Features

* [O] sales - 제품의 일별 판매량  => **Target**
* [X] ID - 제품 ID
* [X] 제품 - 제품 코드
* [O] 대분류 - 제품의 대분류
* [O] 중분류 - 제품의 중분류
* [O] 소분류 - 제품의 소분류
* [O] 브랜드 - 제품의 브랜드
* [X] date - 제품의 판매 날짜
    *  `23.02.23 ~ 23.03.28` : 약 92.65%의 상품이 이 기간동안 0임을 알 수 있음
* [O] quarter - 제품의 판매 분기 (1, 2, 3, 4)분기 존재
* [O] day_name - 제품의 판매 요일
* [O] keyword - 정규화된 제품 브랜드의 키워드 언급 횟수 : 브랜드의 인지도로 판단
* [O] price - 제품의 판매 가격(₩)
* [O] event - 해당 날짜에 event가 있음 (binary? or Category?)


In [5]:
new_train = pd.read_csv(LOADPATH + 'train_fe.csv', low_memory=False)
new_train = new_train.sort_values(by = ['ID', 'date']).reset_index(drop = True)
origin_train = pd.read_csv(PATH + 'train.csv')

In [6]:
def data_FE(df):
    one = df.query('event != "0"')
    one.event = np.ones(len(one), dtype = np.int16)
    df.loc[one.index, 'event'] = one.event
    df.event = df.event.astype(np.int16)

    drop_date = []
    df_enc = df.copy()
    columns = ['ID', '대분류', '중분류', '소분류', '브랜드', 'day_name', 'quarter', 'keyword', 'event', 'sales', 'date']

    for i in range(34):
        drop_date.append(list(np.array(pd.date_range('2023-02-23', periods = 34)).astype(str))[i].split('T')[0])

    for date in tqdm(drop_date):
        drop_idx = df_enc.query('date == @date').index
        df_enc.drop(drop_idx, axis = 0, inplace = True)

    df_enc.drop(['제품','year', 'month', 'day', 'day_of_week', 'price'], axis = 1, inplace = True)
    df_enc = df_enc[columns]
    df_enc = df_enc.drop_duplicates()
    df_enc = df_enc.reset_index(drop = True)

    return df_enc

train_enc = data_FE(new_train)
train_enc.head(3)

  0%|          | 0/34 [00:00<?, ?it/s]

,ID,대분류,중분류,소분류,브랜드,day_name,quarter,keyword,event,sales,date
0,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,Saturday,1,0.84131,1,0,2022-01-01
1,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,Sunday,1,0.91383,0,0,2022-01-02
2,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,Monday,1,1.45053,0,0,2022-01-03


In [7]:
# Label Encoding

col = ['대분류', '중분류', '소분류', '브랜드', 'day_name']
encoder = LabelEncoder()

for c in col:
    train_enc[c] = encoder.fit_transform(train_enc[c])

train_enc.head(3)

,ID,대분류,중분류,소분류,브랜드,day_name,quarter,keyword,event,sales,date
0,0,1,6,37,0,2,1,0.84131,1,0,2022-01-01
1,0,1,6,37,0,3,1,0.91383,0,0,2022-01-02
2,0,1,6,37,0,1,1,1.45053,0,0,2022-01-03


## Make Dataset

### Dataset use base model

In [ ]:
#  시간이 너무 오래걸림
## 현실적으로 쓸 수 없는 함수
## pandas 라이브러리가 너무 무거운 거로 판단됨 -> numpy 데이터로 변형 후 함수 적용하는게 맞는듯
## numpy로 바꿔도 차이가 없음 : 시간복잡도가 너무 높아서 생기는 문제로 판단됨 O(n^2)
## pandas.DataFrame.query()의 문제라고 판명
## ID및 date의 순서로 되어있기 때문에 iloc을 사용해 순서대로 잘라서 dataset을 만들어서 작업 소요시간이 매우 줄음

def make_train_data(data, train_size = CFG['TRAIN_WINDOW_SIZE'], predict_size = CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : date를 melt시킨 새로운 train data
    train_size : 학습에 활용할 기간 => 90 Days
    predict_size : 추론할 기간 => 21 Days
    '''
    window_size = train_size + predict_size         # 90 + 21 = 111
    num_id = data.ID.nunique()                      # 15890
    num_date = data.date.nunique()                  # 425
    num_features = len(data.iloc[0, 1:9])           # 대분류 ~ sales까지
    data = np.array(data)                           # DataFrame to Numpy Data
    
    input_data = np.empty((num_id * ((num_date + num_features) - window_size + 1), train_size, num_features + 1), dtype = np.float16)
    target_data = np.empty((num_id * ((num_date + num_features) - window_size + 1), predict_size), dtype = np.float16)

    for id in tqdm(range(num_id)):
        for j in range(num_date - window_size + 1):      # 315
            temp_data = data[id*425: 425*(id+1)][j:train_size+j, 1:10]
            input_data[id * ((num_date + num_features) - window_size + 1) + j] = temp_data
            target_data[id * ((num_date + num_features) - window_size + 1) + j] = data[id*425: 425*(id+1)][train_size+j:window_size+j, 9] # sales

    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']): #90
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : date를 melt시킨 새로운 train data
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_id = data.ID.nunique()
    num_date = data.date.nunique()
    num_features = data.iloc[0:1, 1:9].shape[1]   # 대분류 ~ sales까지
    data = np.array(data)
    
    test_input = np.empty((num_id, train_size, num_features + 1), dtype = np.float16)

    for id in tqdm(range(num_id)):
        temp_data = data[id*425: 425*(id+1)][-train_size:, 1:10]
        test_input[id] = temp_data

    return test_input

In [ ]:
train_input, train_target = make_train_data(train_enc)
test_input = make_predict_data(train_enc)

In [ ]:
# np.save(PATH + 'train_input', train_input)
# np.save(PATH + 'train_target', train_target)
# np.save(PATH + 'test_input', test_input)

### Another Trainset use Time_slide

In [8]:
# def train_test_split(df):
# list(np.array(pd.date_range('2023-02-23', periods = 34)))

test_df_enc = train_enc[-1:-2]
test_date = train_enc[train_enc.ID == 0][-91:-1].date.to_list()

for date in tqdm(test_date):
    test_smp = train_enc.query('date == @date')
    test_df_enc = pd.concat([test_df_enc, test_smp])

train_df_enc = train_enc.drop(test_df_enc.index, axis = 0).reset_index(drop = True)
test_df_enc = test_df_enc.reset_index(drop = True)

  0%|          | 0/90 [00:00<?, ?it/s]

In [14]:
window_size = 90
forcast_size = 21
date = 'date'
target = 'sales'

def target_standardization(train_df, test_df, target):
    train_df_ = train_df.copy()
    test_df_ = test_df.copy()
    mean_list = []
    std_list = []


    mean, std = np.mean(list(train_df_enc[target])), np.std(list(train_df_enc[target]))

    train_df_.loc[:, target] = (train_df_[target] - mean) / std
    test_df_.loc[:, target] = (test_df_[target] - mean) / std
    return train_df_, test_df_, mean, std


def time_slide_df(df, window_size, forcast_size, date, target):
    df_ = df.copy()
    data_list = []
    dap_list = []
    date_list = []

    for idx in tqdm(range(0, df_.shape[0]-window_size-forcast_size+1)):
        x = df_.loc[idx:idx+window_size-1, target].values.reshape(window_size, 1)       # 90일치 판매량
        y = df_.loc[idx+window_size:idx+window_size+forcast_size-1, target].values      # 21일치 판매량 (target)
        date_ = df_.loc[idx+window_size:idx+window_size+forcast_size-1, date].values    # 날짜
        data_list.append(x)
        dap_list.append(y)
        date_list.append(date_)

    return np.array(data_list, dtype='float16'), np.array(dap_list, dtype='float16'), np.array(date_list)

In [15]:
train_df_enc, test_df_enc, mean_, std_ = target_standardization(train_df_enc, test_df_enc, target)
train_input, train_target, test_date = time_slide_df(train_df_enc, window_size, forcast_size, date, target)

  0%|          | 0/5323040 [00:00<?, ?it/s]

## Base Train

In [6]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간 => 90 Days
    predict_size : 추론할 기간 => 21 Days
    '''
    num_rows = len(data) # 15890
    window_size = train_size + predict_size # 90 + 21 = 111
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1), dtype = np.float16)
    # (5609170, 90, 5)
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size), dtype = np.float16)
    # (5640950, 21)

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # Label: 대분류, 중분류, 소분류, 브랜드
        sales_data = np.array(data.iloc[i, 4:]) # 날짜 데이터: 2022-01-01 ~ 2023-04-04
        
        for j in range(len(sales_data) - window_size + 1): # 0 ~ 348
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [7]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']): #90
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1), dtype = np.float16)
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [8]:
class IndividualMinMaxScaler:
    def __get_min_val(self, df, date_cols):
        df_mins = df[date_cols].min(axis=1).to_numpy()
        return df_mins

    def __get_max_val(self, df, date_cols):
        df_maxs = df[date_cols].max(axis=1).to_numpy()
        return df_maxs


    def fit(self, df: pd.DataFrame):
        date_cols = [col for col in df.columns  # 일자 변수들만 Scale하기
                     if col.startswith("2")]

        self.min_val = self.__get_min_val(df, date_cols)
        self.max_val = self.__get_max_val(df, date_cols)
        denom = self.max_val - self.min_val
        self.denom = np.where(denom==0, 1, denom)

    def transform(self, df: pd.DataFrame):
        date_cols = [col for col in df.columns
                     if col.startswith("2")]
        return df[date_cols] \
            .apply(lambda x: (x-self.min_val)/self.denom)

    def fit_transform(self, df: pd.DataFrame):
        self.fit(df)
        return self.transform(df)

In [9]:
train_scale = origin_train.copy()

train_scale.drop(['ID', '제품'], axis = 1, inplace = True)
scaler = IndividualMinMaxScaler()
train_scale.iloc[:, 4:] = scaler.fit_transform(origin_train)
print(min(train_scale.iloc[:, 6]), max(train_scale.iloc[:, 6]))
train_scale.head(3)

0.0 1.0


,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [10]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_scale[col])
    train_scale[col] = label_encoder.transform(train_scale[col])

In [11]:
train_input, train_target = make_train_data(train_scale)
test_input = make_predict_data(train_scale)

# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

print(train_input.shape)
print(train_target.shape)
print(val_input.shape)
print(val_target.shape)
print(test_input.shape)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

(4487336, 90, 5)
(4487336, 21)
(1121834, 90, 5)
(1121834, 21)
(15890, 90, 5)


## Loss

#### Scaler

In [ ]:
class IndividualMinMaxScaler:
    def __get_min_val(self, df, date_cols):
        df_mins = df[date_cols].min(axis=1).to_numpy()
        return df_mins

    def __get_max_val(self, df, date_cols):
        df_maxs = df[date_cols].max(axis=1).to_numpy()
        return df_maxs


    def fit(self, df: pd.DataFrame):
        date_cols = [col for col in df.columns  # 일자 변수들만 Scale하기
                     if col.startswith("2")]

        self.min_val = self.__get_min_val(df, date_cols)
        self.max_val = self.__get_max_val(df, date_cols)
        denom = self.max_val - self.min_val
        self.denom = np.where(denom==0, 1, denom)

    def transform(self, df: pd.DataFrame):
        date_cols = [col for col in df.columns
                     if col.startswith("2")]
        return df[date_cols] \
            .apply(lambda x: (x-self.min_val)/self.denom)

    def fit_transform(self, df: pd.DataFrame):
        self.fit(df)
        return self.transform(df)

#### Metric

In [ ]:
def pseudo_sfa(pred, df):
    pred_length = pred.shape[1] - 1
    true = df.iloc[:, -pred_length:].reset_index() \
        .rename(columns={"index": "ID"})

    main_id = {}
    for main_cat in df["대분류"].unique():
        main_id[main_cat] = df.query("대분류==@main_cat")["ID"].to_list()

    psfa = []
    for main_cat in main_id.keys():
        indices = true["ID"].isin(main_id[main_cat])

        true_arr = true[indices].iloc[:, 1:].to_numpy()
        pred_arr = pred[indices].iloc[:, 1:].to_numpy()

        eps = np.ones((true_arr.shape)) / 1e8

        true_sum = true_arr.sum(axis=0)
        true_sum = np.stack([true_sum]*len(true_arr)) + eps
        true_rate = true_arr / true_sum

        abs_error = np.abs(true_arr - pred_arr)
        denom = np.maximum(true_arr, pred_arr+eps)
        
        score = 1 - (1 / true_arr.shape[1]
                     * (abs_error / denom) * true_rate).sum()
        psfa.append(score)
        print(main_cat, score)

    return np.mean(psfa)

#### Loss Function

In [ ]:
class PsfaLoss(nn.Module):
    def __init__(self, scaler, df):
        super().__init__()
        self.scaler = scaler
        self.main_cats = df.groupby("대분류")["ID"].unique().values
    
    def forward(self, pred, true):
        # pred: [batch_size, length, products(15890)]
        pred = pred * torch.tensor(self.scaler.denom) \
            + torch.tensor(self.scaler.min_val)
        true = true * torch.tensor(self.scaler.denom) \
            + torch.tensor(self.scaler.min_val)

        L1scaled = torch.abs(true-pred) / torch.maximum(pred, true+1e-8)
        
        rate = torch.zeros_like(true)
        for i in range(len(self.main_cats)):
            rate[:, :, self.main_cats[i]] = \
                true[:, :, self.main_cats[i]] \
                / (true[:, :, self.main_cats[i]].sum(dim=-1, keepdim=True) + 1e-8) \
                / len(self.main_cats)
        return (L1scaled * rate).sum() / (true.shape[0] * true.shape[1])

## DLinear Model

In [17]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

# Train / Validation Split
data_len = len(train_input)
train_dataset = CustomDataset(train_input[:-int(data_len*0.2)], train_target[:-int(data_len*0.2)])
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(train_input[-int(data_len*0.2):], train_target[-int(data_len*0.2):])
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# # Feature Engineering한 Data

# train_input = np.load(LOADPATH + 'train_input.npy')
# train_target = np.load(LOADPATH + 'train_target.npy')
# test_input = np.load(LOADPATH + 'test_input.npy')

In [18]:
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size = 25, stride = 1):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
    	# [BATCH SIZE, SEQ_LEN, CHANNEL]
        # padding on the both ends of time series

        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x # [BATCH SIZE, SEQ_LEN, CHANNEL]

In [19]:
class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [20]:
# train_input.shape = (data_num, window_size, feature_size)
# 

class DLinear(torch.nn.Module):
    """
    Decomposition-Linear
    """
    def __init__(self, window_size = CFG['TRAIN_WINDOW_SIZE'], forcast_size=CFG['PREDICT_SIZE'], kernel_size = 25, individual = False, feature_size=1):
        super(DLinear, self).__init__()
        self.window_size = window_size
        self.forcast_size = forcast_size

        # Decomposition Kernel Size
        self.decompsition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = feature_size
        
        if self.individual:
            self.Linear_Seasonal = torch.nn.ModuleList().to(device)
            self.Linear_Trend = torch.nn.ModuleList().to(device)
            
            for i in range(self.channels):
                self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forcast_size).to(device))
                self.Linear_Trend[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size])).to(device)

                # Use this two lines if you want to visualize the weights
                self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Seasonal[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

        else:
            self.Linear_Trend = torch.nn.Linear(self.window_size, self.forcast_size).to(device)
            self.Linear_Trend.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

            # Use this two lines if you want to visualize the weights
            self.Linear_Seasonal = torch.nn.Linear(self.window_size,  self.forcast_size).to(device)
            self.Linear_Seasonal.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

    def forward(self, x):
        # x: [Batch, Input length, Channel]
        trend_init, seasonal_init = self.decompsition(x)
        trend_init, seasonal_init = trend_init.permute(0,2,1), seasonal_init.permute(0,2,1)

        if self.individual:
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forcast_size], dtype=trend_init.dtype).to(trend_init.device)
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forcast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
            
            for idx in range(self.channels):
                trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :]).to(device)
                seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :]).to(device)

        else:
            trend_output = self.Linear_Trend(trend_init).to(device)
            seasonal_output = self.Linear_Seasonal(seasonal_init).to(device)
        
        x = seasonal_output + trend_output
        
        return x.permute(0,2,1)

## Train

In [21]:
train_loss_li = []
val_loss_li = []
test_loss_li = []
epoch = 10
lr = 1e-6

DLinear_model = DLinear(
    window_size = CFG['TRAIN_WINDOW_SIZE'],
    forcast_size=CFG['PREDICT_SIZE'],
    kernel_size = 25,
    individual = False,
    feature_size = train_dataset.X.shape[2]
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(DLinear_model.parameters(), lr = lr)
best_loss = 9999999

In [22]:
for epoch in tqdm(range(1, 11)):
    loss_list = []
    DLinear_model.train()

    for data, target in tqdm(iter(train_loader)):
        data = data.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()

        output = DLinear_model(data)
        loss = criterion(output, target.unsqueeze(-1))

        loss.backward()
        optimizer.step()

        loss_list.append(loss.item())    

    train_loss_li.append(np.mean(loss_list))

    DLinear_model.eval()
    with torch.no_grad():
        for data, target in tqdm(iter(val_loader)):
            data = data.to(device)
            target = target.to(device)

            output = DLinear_model(data)
            valid_loss = criterion(output, target.unsqueeze(-1))

            val_loss_li.append(valid_loss.item())

    print(f'Epoch : [{epoch}] Train Loss : [{np.mean(loss_list):.4f}] Val Loss : [{np.mean(val_loss_li):.4f}]')

    if valid_loss < best_loss:
        # torch.save(DLinear_model, 'DLinear_model.pth')
        best_loss = valid_loss
        dlinear_best_epoch = epoch
        dlinear_best_train_loss = np.mean(loss_list)
        dlinear_best_valid_loss = np.mean(valid_loss.item())
        print("Model Saved")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/66538 [00:00<?, ?it/s]

  0%|          | 0/16635 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.9823] Val Loss : [0.1922]
Model Saved


  0%|          | 0/66538 [00:00<?, ?it/s]

...

  0%|          | 0/16635 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.9379] Val Loss : [0.1883]
Model Saved


  0%|          | 0/66538 [00:00<?, ?it/s]

  0%|          | 0/16635 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.9363] Val Loss : [0.1881]
Model Saved


In [25]:
time_slide_df(test_df_enc, window_size, 0, date, target)


  0%|          | 0/1430011 [00:00<?, ?it/s]

ValueError: Cannot index with multidimensional key

In [23]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

NameError: name 'test_input' is not defined

In [22]:
predictions = []

with torch.no_grad():
    for X in tqdm(iter(test_loader)):
        X = X.to(device)

        output = DLinear_model(X)
        # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
        output = output.cpu().numpy()
            
        predictions.extend(output)

  0%|          | 0/249 [00:00<?, ?it/s]

In [29]:
np.array(predictions)

array([[0.01744074, 0.03097999, 0.0335838 , ..., 0.03069846, 0.03145359,
        0.0432485 ],
       [0.0475485 , 0.04547943, 0.06210718, ..., 0.05769744, 0.05753664,
        0.06239583],
       [0.00410482, 0.00686081, 0.00874485, ..., 0.01962999, 0.02041792,
        0.02107154],
       ...,
       [0.00361981, 0.00678486, 0.0090176 , ..., 0.01900744, 0.01884934,
        0.0187877 ],
       [0.10489664, 0.08854678, 0.06268504, ..., 0.04553596, 0.04296607,
        0.04558872],
       [0.00367289, 0.00620643, 0.00812227, ..., 0.01828414, 0.01888015,
        0.01933588]], dtype=float32)

In [31]:
prediction = np.array(predictions)

submit = pd.read_csv(PATH + '/sample_submission.csv') # submit은 15690개 상품 / 예측은 15682개 상품
submit.iloc[:, 1:] = prediction

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0.017441,0.030980,0.033584,0.022638,0.013365,0.017369,0.030133,0.039729,0.050789,...,0.024773,0.025457,0.034845,0.044702,0.053858,0.051216,0.037808,0.030698,0.031454,0.043249
1,1,0.047549,0.045479,0.062107,0.057356,0.051628,0.053787,0.059697,0.069866,0.076554,...,0.057258,0.057448,0.063567,0.072180,0.078113,0.073422,0.064179,0.057697,0.057537,0.062396
2,2,0.004105,0.006861,0.008745,0.009651,0.010667,0.011621,0.012523,0.013447,0.014126,...,0.015659,0.016318,0.017067,0.017778,0.018308,0.018709,0.019102,0.019630,0.020418,0.021072
3,3,0.003673,0.006206,0.008122,0.009051,0.009837,0.010603,0.011314,0.012041,0.012788,...,0.014462,0.014986,0.015574,0.016197,0.016808,0.017362,0.017839,0.018284,0.018880,0.019336
4,4,0.003771,0.009547,0.016228,0.020355,0.017405,0.012639,0.010349,0.011653,0.016810,...,0.022983,0.018428,0.016688,0.017992,0.021929,0.027962,0.030493,0.026933,0.022235,0.019752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,-0.012594,-0.006994,0.011467,0.035730,0.045422,0.029446,0.006225,-0.005651,-0.000057,...,0.054787,0.040858,0.018686,0.005335,0.008692,0.030258,0.053982,0.058707,0.041828,0.020154
15886,15886,0.026855,0.035373,0.039040,0.037397,0.029384,0.022271,0.021897,0.027614,0.036263,...,0.035879,0.028235,0.028987,0.036884,0.046752,0.053272,0.051612,0.042663,0.034726,0.032476
15887,15887,0.003620,0.006785,0.009018,0.010113,0.010578,0.010671,0.010959,0.011858,0.013141,...,0.015151,0.014812,0.015046,0.015891,0.017131,0.018109,0.018912,0.019007,0.018849,0.018788
15888,15888,0.104897,0.088547,0.062685,0.051906,0.045335,0.041895,0.044290,0.048533,0.053316,...,0.045509,0.043063,0.045253,0.049968,0.054557,0.055319,0.051415,0.045536,0.042966,0.045589


In [32]:
submit.to_csv(PATH + 'Dlinear_submit_1.csv', index_label=False)